In [1]:
import pandas as pd
import sys
sys.path.append("/Users/officialbiznas/Documents/GitHub/stockFries/app")
import scraper
import config
import json
import os
import shutil

In [2]:
%%time
scraper.pandas_analysis(config.fund_dict)

2021-01-28 00:36:02.735509  :: Scrpaing ::  ['https://www.sec.gov/Archives/edgar/data/1358706/000117266120002136/xslForm13F_X01/infotable.xml', 'https://www.sec.gov/Archives/edgar/data/1358706/000117266120001661/xslForm13F_X01/infotable.xml']
2021-01-28 00:36:07.039665  :: Scrpaing ::  ['https://www.sec.gov/Archives/edgar/data/1112520/000111252020000024/xslForm13F_X01/infotable.xml', 'https://www.sec.gov/Archives/edgar/data/1112520/000111252020000022/xslForm13F_X01/infotable.xml']
2021-01-28 00:36:11.181367  :: Scrpaing ::  ['https://www.sec.gov/Archives/edgar/data/1491072/000131586320000947/xslForm13F_X01/ansonfunds3q20.inftab.xml', 'https://www.sec.gov/Archives/edgar/data/1491072/000131586320000688/xslForm13F_X01/ansonfunds2q20.inftab.xml']
2021-01-28 00:36:12.212260  :: Scrpaing ::  ['https://www.sec.gov/Archives/edgar/data/1656456/000165645620000004/xslForm13F_X01/Form13FInfoTable.xml', 'https://www.sec.gov/Archives/edgar/data/1656456/000165645620000003/xslForm13F_X01/Form13FInfoTa

In [15]:
with open(config.scrapped_json_fn_options, 'r') as f:
    hedge_fund_json_dg = json.load(f)
df_recent_options = pd.read_json(hedge_fund_json_dg, orient='split')
df_puts_only = df_recent_options.drop_duplicates(subset=['CUSIP','HF_Name'], keep=False)
df_puts_only = df_puts_only[(df_puts_only['Option_Type']=='Put') & (df_puts_only['HF_Name']!='CAPITAL FUND MANAGEMENT')]
print(len(set(df_puts_only.HF_Name.tolist())))
df_puts_only.sort_values('Company').reset_index().tail(100)
df_puts_only.groupby(['CUSIP','Company','Option_Type'],as_index=False).agg(
        {'Shares': 'sum', 'Value': 'sum', 'HF_Name': 'unique'}).sort_values('Value').tail(20)

15


,CUSIP,Company,Option_Type,Shares,Value,HF_Name
52,896522109,TRINITY INDS INC,Put,1350000,26325,[Melvin Capital Management LP]
18,46284V101,IRON MTN INC NEW,Put,1010000,27058,[Melvin Capital Management LP]
50,89154L950,TORTOISE ACQUISITION CORP,Put,525000,28093,[HIGHBRIDGE CAPITAL MANAGEMENT LLC]
4,136069101,CANADIAN IMP BK COMM,Put,400000,29888,[POLAR ASSET MANAGEMENT PARTNERS]
32,681116109,OLLIES BARGAIN OUTLET HLDGS,Put,390000,34067,[Melvin Capital Management LP]
5,23918K108,DAVITA INC,Put,400000,34260,"[GATES CAPITAL MANAGEMENT, INC]"
17,462726100,IROBOT CORP,Put,500000,37950,[Melvin Capital Management LP]
7,28414H103,ELANCO ANIMAL HEALTH INC,Put,1670000,46643,[Melvin Capital Management LP]
30,635017106,NATIONAL BEVERAGE CORP,Put,800000,54408,[Melvin Capital Management LP]
11,36467W109,GAMESTOP CORP NEW,Put,5400000,55080,[Melvin Capital Management LP]


In [7]:
with open(config.scrapped_json_fn_hedgefund, 'r') as f:
    hedge_fund_json_dg = json.load(f)
df_recent_filing = pd.read_json(hedge_fund_json_dg, orient='split').T.rename(columns={0:'link',1:'recent_filing_date',2:'current_holdings',3:'previous_holdings'})
df_recent_filing_head = df_recent_filing [df_recent_filing.recent_filing_date == df_recent_filing.recent_filing_date.max()].head(1)
most_recent_filing_date = df_recent_filing_head.recent_filing_date.tolist()[0]
most_recent_filing_url = df_recent_filing_head.link.tolist()[0]
most_recent_filing_fund = df_recent_filing_head.index.tolist()[0]

,link,recent_filing_date,current_holdings,previous_holdings
Mangrove Partners,https://www.sec.gov/cgi-bin/browse-edgar?actio...,2020-11-16,"[{'CUSIP': '03076C106', 'Company': 'AMERIPRISE...","[{'CUSIP': '03076C106', 'Company': 'AMERIPRISE..."
Segantii,https://www.sec.gov/cgi-bin/browse-edgar?actio...,2020-11-10,"[{'CUSIP': 'G0750C108', 'Company': 'AXALTA COA...","[{'CUSIP': 'G0750C108', 'Company': 'AXALTA COA..."
MIG Capital,https://www.sec.gov/cgi-bin/browse-edgar?actio...,2020-11-16,"[{'CUSIP': '00790R104', 'Company': 'ADVANCED D...","[{'CUSIP': '00790R104', 'Company': 'ADVANCED D..."
Masters Capital Management,https://www.sec.gov/cgi-bin/browse-edgar?actio...,2020-11-16,"[{'CUSIP': '88579Y101', 'Company': '3M CO', 'V...","[{'CUSIP': '88579Y101', 'Company': '3M CO', 'V..."
Harvest Capital Strategies,https://www.sec.gov/cgi-bin/browse-edgar?actio...,2019-11-14,"[{'CUSIP': '09180C106', 'Company': 'BJS RESTAU...","[{'CUSIP': '09180C106', 'Company': 'BJS RESTAU..."


In [11]:
with open(config.scrapped_json_fn, 'r') as f:
    json_df = json.load(f)
df = pd.read_json(json_df, orient='split')
df.sort_values('Company').head()

,cusip,Company,Value_x,Shares_x,Ownership,Value_y,Shares_y,Change,ticker,URL_Yahoo
1728,000000000,-,0.0,0.0,0.0,NaN,NaN,NaN,None,https://finance.yahoo.com/quote/None?p=None
1729,000000000,0,NaN,NaN,NaN,0.0,0.0,NaN,None,https://finance.yahoo.com/quote/None?p=None
1340,88025U109,10X Genomics Inc,5805.0,65000.0,0.0,NaN,NaN,NaN,TXG,https://finance.yahoo.com/quote/TXG?p=TXG
2261,68269G107,1Life Healthcare Inc,NaN,NaN,NaN,4831.0,4500000.0,NaN,ONEM,https://finance.yahoo.com/quote/ONEM?p=ONEM
2470,90138A953,21Vianet Group Inc Put,NaN,NaN,NaN,2633.0,190000.0,NaN,None,https://finance.yahoo.com/quote/None?p=None


,link,recent_filing_date,current_holdings,previous_holdings
Mangrove Partners,https://www.sec.gov/cgi-bin/browse-edgar?actio...,2020-11-16,"[{'CUSIP': '03076C106', 'Company': 'AMERIPRISE...","[{'CUSIP': '03076C106', 'Company': 'AMERIPRISE..."
Segantii,https://www.sec.gov/cgi-bin/browse-edgar?actio...,2020-11-10,"[{'CUSIP': 'G0750C108', 'Company': 'AXALTA COA...","[{'CUSIP': 'G0750C108', 'Company': 'AXALTA COA..."
MIG Capital,https://www.sec.gov/cgi-bin/browse-edgar?actio...,2020-11-16,"[{'CUSIP': '00790R104', 'Company': 'ADVANCED D...","[{'CUSIP': '00790R104', 'Company': 'ADVANCED D..."
Masters Capital Management,https://www.sec.gov/cgi-bin/browse-edgar?actio...,2020-11-16,"[{'CUSIP': '88579Y101', 'Company': '3M CO', 'V...","[{'CUSIP': '88579Y101', 'Company': '3M CO', 'V..."
Harvest Capital Strategies,https://www.sec.gov/cgi-bin/browse-edgar?actio...,2019-11-14,"[{'CUSIP': '09180C106', 'Company': 'BJS RESTAU...","[{'CUSIP': '09180C106', 'Company': 'BJS RESTAU..."
Anson Funds,https://www.sec.gov/cgi-bin/browse-edgar?actio...,2020-11-16,"[{'CUSIP': '00507V109', 'Company': 'ACTIVISION...","[{'CUSIP': '00507V109', 'Company': 'ACTIVISION..."
Capital Fund Management,https://www.sec.gov/cgi-bin/browse-edgar?actio...,2020-11-13,"[{'CUSIP': '12572Q105', 'Company': 'CME GROUP ...","[{'CUSIP': '12572Q105', 'Company': 'CME GROUP ..."
Hawk Ridge Capital Management,https://www.sec.gov/cgi-bin/browse-edgar?actio...,2020-11-16,"[{'CUSIP': '002535300', 'Company': 'AARONS INC...","[{'CUSIP': '002535300', 'Company': 'AARONS INC..."
Contrarian Capital Management,https://www.sec.gov/cgi-bin/browse-edgar?actio...,2020-11-16,"[{'CUSIP': '04650Y100', 'Company': 'AT HOME GR...","[{'CUSIP': '04650Y100', 'Company': 'AT HOME GR..."
Dorsal Capital Management,https://www.sec.gov/cgi-bin/browse-edgar?actio...,2020-11-16,"[{'CUSIP': '023135106', 'Company': 'AMAZON COM...","[{'CUSIP': '023135106', 'Company': 'AMAZON COM..."


In [73]:
filings_by_fund = df_recent_filing.reset_index().rename(columns={'index':'fund_name'}) \
                    [['fund_name','recent_filing_date','current_holdings','previous_holdings','link']
                    ].dropna().head(n=50).values.tolist()

#list_of_invested_stocks = []
for i, fund_filing in enumerate(filings_by_fund):
    filings_by_fund_stocksOnly = [x['Company'] for x in fund_filing[2]]
    #list_of_invested_stocks.append(filings_by_fund_stocksOnly)
    filings_by_fund[i].insert(6, filings_by_fund_stocksOnly)
print(len(filings_by_fund[0]))

6


In [72]:
filings_by_fund[0]
filings_by_fund_stocksOnly
list_of_invested_stocks[4]
filings_by_fund[2]

['MIG Capital',
 '2020-11-16',
 [{'CUSIP': '00790R104',
   'Company': 'ADVANCED DRAIN SYS INC DEL',
   'Value': 56257,
   'Shares': 900971},
  {'CUSIP': '02079K107',
   'Company': 'ALPHABET INC',
   'Value': 41926,
   'Shares': 28529},
  {'CUSIP': '03768E105',
   'Company': 'APOLLO GLOBAL MGMT INC',
   'Value': 43902,
   'Shares': 981046},
  {'CUSIP': '04247X102',
   'Company': 'ARMSTRONG WORLD INDS INC',
   'Value': 46610,
   'Shares': 677378},
  {'CUSIP': '058498106',
   'Company': 'BALL CORP',
   'Value': 47143,
   'Shares': 567169},
  {'CUSIP': '101388106',
   'Company': 'BOTTOMLINE TECH DEL INC',
   'Value': 45453,
   'Shares': 1078105},
  {'CUSIP': '19239V302',
   'Company': 'COGENT COMMUNICATIONS HLDGS',
   'Value': 58938,
   'Shares': 981480},
  {'CUSIP': '237194105',
   'Company': 'DARDEN RESTAURANTS INC',
   'Value': 37218,
   'Shares': 369442},
  {'CUSIP': '31620M106',
   'Company': 'FIDELITY NATL INFORMATION SV',
   'Value': 35164,
   'Shares': 238873},
  {'CUSIP': '31847R1

In [29]:
filings_by_fund = df_recent_filing.reset_index().rename(columns={'index':'fund_name'}) \
                    [['fund_name','recent_filing_date','current_holdings','previous_holdings','link']
                 ].dropna().head(n=50).values.tolist()
#print(filings_by_fund[0])
print(len(filings_by_fund[0]))

5


In [22]:
with open(config.scrapped_json_fn, 'r') as f:
        json_df = json.load(f)
df = pd.read_json(json_df, orient='split')
try:
    ownership = df[['Company', 'Ownership','URL_Yahoo']
                 ].dropna().head(n=20).values.tolist()
except:
    ownership = df[['Company', 'Ownership']
                 ].dropna().head(n=20).values.tolist()
print(ownership)

[['Tesla Inc', 2.94, 'https://finance.yahoo.com/quote/TSLA?p=TSLA'], ['Boeing Co', 2.56, 'https://finance.yahoo.com/quote/BA?p=BA'], ['Netflix Inc', 2.32, 'https://finance.yahoo.com/quote/NFLX?p=NFLX'], ['Apple Inc', 2.1, 'https://finance.yahoo.com/quote/AAPL?p=AAPL'], ['Exxon Mobil Corp', 2.09, 'https://finance.yahoo.com/quote/XOM?p=XOM'], ['Newmont Corp', 1.49, 'https://finance.yahoo.com/quote/NEM?p=NEM'], ['Amgen Inc', 1.4100000000000001, 'https://finance.yahoo.com/quote/AMGN?p=AMGN'], ['Altria Group Inc', 1.25, 'https://finance.yahoo.com/quote/MO?p=MO'], ['Intel Corp', 1.21, 'https://finance.yahoo.com/quote/INTC?p=INTC'], ['Square Inc', 1.19, 'https://finance.yahoo.com/quote/SQ?p=SQ'], ['Alibaba Group Hldg Ltd', 1.17, 'https://finance.yahoo.com/quote/BABA?p=BABA'], ['Facebook Inc', 1.11, 'https://finance.yahoo.com/quote/FB?p=FB'], ['Spdr Ser Tr', 1.1, 'https://finance.yahoo.com/quote/XOP?p=XOP'], ['Spdr S&P 500 Etf Tr', 1.09, 'https://finance.yahoo.com/quote/SPY?p=SPY'], ['Wells Fa

In [81]:
try: #Copy over files to static folder for downloading
    to_static_fn_byHF = os.path.join(os.path.dirname(os.getcwd()),
        'static',os.path.basename(config.scrapped_json_fn_hedgefund))
    print(to_static_fn_byHF)
    shutil.copyfile(config.scrapped_json_fn_hedgefund, to_static_fn_byHF)
except Exception as e:
    print('Error in file Copy: ', e, flush=True)

/Users/officialbiznas/Documents/GitHub/stockFries/static/json_df_dataByHedgeFund.txt
Error in file Copy:  [Errno 2] No such file or directory: '/Users/officialbiznas/Documents/GitHub/stockFries/static/json_df_dataByHedgeFund.txt'


In [78]:
to_static_fn_byHF = os.path.join(os.path.dirname(os.getcwd()),
        'static',os.path.basename(config.scrapped_json_fn_hedgefund))